## Carregamento dos Dados

In [9]:
import tensorflow as tf

x_train = tf.keras.utils.image_dataset_from_directory(
    "dataset/train",
    labels="inferred", # Os rótulos são definidos a partir dos nomes das subpastas
    label_mode="categorical", # Formato categórico dos rótulos
    batch_size=32, # Lotes de 32 imagens
    image_size=(128, 128), # Redimensiona as imagens para 128x128
    shuffle=True, # Embaralha as imagens para evitar que a rede aprenda padrões de ordenação
)

x_test = tf.keras.utils.image_dataset_from_directory(
    "dataset/test",
    labels="inferred", # Os rótulos são definidos a partir dos nomes das subpastas
    label_mode="categorical", # Formato categórico dos rótulos
    batch_size=32, # Lotes de 32 imagens
    image_size=(128, 128), # Redimensiona as imagens para 128x128
    shuffle=False, # Não precisa embaralhar nos dados de teste
)

x_val = tf.keras.utils.image_dataset_from_directory(
    "dataset/validation",
    labels="inferred", # Os rótulos são definidos a partir dos nomes das subpastas
    label_mode="categorical", # Formato categórico dos rótulos
    batch_size=32, # Lotes de 32 imagens
    image_size=(128, 128), # Redimensiona as imagens para 128x128
    shuffle=False, # Não precisa embaralhar nos dados de validação
)

Found 7000 files belonging to 7 classes.
Found 1400 files belonging to 7 classes.
Found 1400 files belonging to 7 classes.


## Normalização

In [10]:
# Camada que normaliza as imagens do intervalo [0, 255] para [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Aplicação a normalização às imagens de cada dataset
x_train = x_train.map(lambda x, y: (normalization_layer(x), y))
x_val = x_val.map(lambda x, y: (normalization_layer(x), y))
x_test = x_test.map(lambda x, y: (normalization_layer(x), y))

## Criação do Modelo CNN

In [11]:
# Modelo sequencial onde as camadas são empilhadas uma acima da outra
model = tf.keras.Sequential([
    
    # Primeira camada Convolucional 
    tf.keras.layers.Conv2D(32, # Número de filtros
                           (3, 3), # Tamanho do filtro (3 por 3 pixels)
                           activation='relu', # Função de ativação ReLU
                           input_shape=(128, 128, 3)), # Dimenções das imagens

    # Reduz a resolução da imagem pela metade através do agrupamento guloso de um grupo de 2 por 2 pixels
    tf.keras.layers.MaxPooling2D((2, 2)), 

    # Segunda camada Convolucional 
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Terceira camada Convolucional 
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Transforma a saída 3D da ultima camada em um vetor de uma dimenção
    tf.keras.layers.Flatten(),
    
    # Evita overfitting desligando 50% dos neurónios de maneira aleatória durante os treinos
    tf.keras.layers.Dropout(0.5),

    # Camada com 128 neurônios conectados
    tf.keras.layers.Dense(128, activation='relu'),
    
    # Camada com o numero de tipos de celulas e função de ativação Softmax (transforma valores em probabilidades)
    tf.keras.layers.Dense(7, activation='softmax')
])

## Compilação do Modelo

In [12]:
# Compila o modelo deixado pronto para treino
model.compile(
    # Otimizador 
    optimizer='adam',
    # Função de perda
    loss='categorical_crossentropy',
    # Métrica para acompanhar a acurácia. 
    metrics=['accuracy']
)
